In [1]:
from plot_params import *
import pandas as pd
import random
import string
from tqdm.notebook import trange
import os
df=pd.read_csv(outdir / 'Human_annotated_30bp.csv.gz')

In [2]:
df['subseq'].str.len().min()

30

In [3]:
df['subseq'].str.len().max()

30

In [4]:
df.head()

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq
0,0,ENST00000000233,0,30,0.000000,0.044434,UTR5,UTR5,CTGCTGCTGCTGCGCCCCATCCCCCCGCGG
1,1,ENST00000000233,15,45,0.000000,0.044434,UTR5,UTR5,CCCATCCCCCCGCGGCCGGCCAGTTCCAGC
2,2,ENST00000000233,30,60,0.000000,0.044434,UTR5,UTR5,CCGGCCAGTTCCAGCCCGCACCCCGCGTCG
3,3,ENST00000000233,45,75,0.000000,0.044434,UTR5,UTR5,CCGCACCCCGCGTCGGTGCCCGCGCCCCTC
4,4,ENST00000000233,60,90,-0.001953,0.042480,UTR5,CDS,GTGCCCGCGCCCCTCCCCGGGCCCCGCCAT


In [5]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq
def df2fasta(df, fname, col='subseq', id='Unnamed: 0'):
    
    
    sequences = []
    for index, row in df.iterrows():
        sequences.append(SeqRecord(Seq(row[col]), str(row[id]), "", ""))
    with open(fname, "w") as output_handle:
        SeqIO.write(sequences, output_handle, "fasta")


In [6]:
outdir

PosixPath('/tscc/nfs/home/bay001/projects/karen_synapse_20240529/permanent_data/charlene_work')

In [7]:
df2fasta(df, outdir / 'all_human_30bp.fa')

In [8]:
df['annotation']=df.apply(
    lambda row: 'UTR5' if row['start_annotation']=='UTR5' and row['end_annotation']=='UTR5' else
     'UTR3' if row['start_annotation']=='UTR3' and row['end_annotation']=='UTR3' else 'CDS', axis = 1)

In [9]:
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']<-0.1)], 
         outdir / 'utr3_forefround_human_30bp.fa')
df2fasta(df.loc[(df['annotation']=='UTR3')&(df['difference']>0.1)], 
         outdir / 'utr3_backfround_human_30bp.fa')

df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)], 
         outdir / 'utr5_forefround_human_30bp.fa')
df2fasta(df.loc[(df['annotation']=='UTR5')&(df['difference']>0.1)], 
         outdir / 'utr5_backfround_human_30bp.fa')

In [10]:
df.loc[(df['annotation']=='UTR5')&(df['difference']<-0.1)]

,Unnamed: 0,transcript_id,deletion_index_start,deletion_index_end,difference,synapse_probability,start_annotation,end_annotation,subseq,annotation
2781,2781,ENST00000005259,0,30,-0.369141,0.279297,UTR5,UTR5,GTCCCGCGCGTCGGCGGCCGCGGAGCAGCG,UTR5
2782,2782,ENST00000005259,15,45,-0.212891,0.435547,UTR5,UTR5,GGCCGCGGAGCAGCGCAGGGAGCCAGGCGG,UTR5
13444,13444,ENST00000035307,1080,1110,-0.185547,0.043945,UTR5,UTR5,GGGGCTGTTGTTTTGATGGATCGTGTGCTT,UTR5
13445,13445,ENST00000035307,1095,1125,-0.195801,0.033691,UTR5,UTR5,ATGGATCGTGTGCTTTTCCCTTACCTCTTA,UTR5
13446,13446,ENST00000035307,1110,1140,-0.138672,0.090820,UTR5,UTR5,TTCCCTTACCTCTTATCACTTGCTGTCATC,UTR5
...,...,...,...,...,...,...,...,...,...,...
6949606,6949606,ENST00000706492,150,180,-0.229492,0.131836,UTR5,UTR5,AGTTGGGATCTCTTTGCTTTTCTGTTTGGC,UTR5
6950081,6950081,ENST00000706533,120,150,-0.462891,0.279297,UTR5,UTR5,CGCCGGCGGGGCTCGGTCCAGTCCTCATGG,UTR5
6961296,6961296,ENST00000706836,360,390,-0.179688,0.464844,UTR5,UTR5,AGCGTGGGAACGCGGCTGGCAGGCTGTGGA,UTR5
6961297,6961297,ENST00000706836,375,405,-0.433594,0.210938,UTR5,UTR5,CTGGCAGGCTGTGGACCTCGTCCTCACCAC,UTR5


# Commands to run HOMER

```
cd /tscc/nfs/home/hsher/ps-yeolab5/karen_data
findMotifs.pl utr3_forefround_human_30bp.fa fasta utr3_homer_results_human/ -fasta utr3_backfround_human_30bp.fa \
    -rna -nofacts
findMotifs.pl utr5_forefround_human_30bp.fa fasta utr5_homer_results_human/ -fasta utr5_backfround_human_30bp.fa \
    -rna -nofacts
```

# Random sample/shuffle sequences to re-score as background

In [11]:
def shuffle(input_string):
    """
    Randomize a string by shuffling its characters using random.shuffle()
    
    Args:
        input_string (str): The original string to randomize
    
    Returns:
        str: Randomized version of the input string
    """
    # Convert string to list of characters
    char_list = list(input_string)
    
    # Shuffle the list in-place
    random.shuffle(char_list)
    
    # Convert back to string
    return ''.join(char_list)

In [12]:
progress = trange(50)
for i in range(50, 100, 1):
    df2 = df.copy()
    try:
        os.makedirs(os.path.join(outdir, 'eclip_model_scores_human_30bp_shuffled', f"iter_{i}"), exist_ok=True)
    except Exception as e:
        print(e)
        raise
    df2['subseq_shuffled'] = df2['subseq'].apply(shuffle)
    df2fasta(df2, fname=os.path.join(outdir, 'eclip_model_scores_human_30bp_shuffled', f"iter_{i}", 'all_human_30bp.fa'), col='subseq_shuffled')
    progress.update(1)

  0%|          | 0/50 [00:00<?, ?it/s]